In [1]:
# Import libraries
# import nltk
# nltk.download("stopwords")
# python -m textblob.download_corpora (instala bases de datos linguisticas)
# python -m spacy download en_core_web_sm (base de datos en ingles)
from textblob import TextBlob
import pandas as pd
import tweet-preprocessor as p
import statistics
from nltk.corpus import stopwords
import csv
import spacy
import numpy as np
import seaborn as sns
nlp = spacy.load("en_core_web_sm")
%matplotlib inline

In [2]:
# Load data
twitterFeed = pd.read_csv(
    "../data/extracted/tweepy/gameofthrones.csv",
    header=None,
    names=["datetime", "tweet"],
    encoding="utf-8"
)
twitterFeed.dtypes

datetime    object
tweet       object
dtype: object

In [4]:
twitterFeed

,datetime,tweet
0,2019-06-02 00:22:56,RT @isyuniarto: Which House are you?\nGOTZILLA...
1,2019-06-02 00:22:54,My 🔥#GameofThrones #Finale take: it was fuckin...
2,2019-06-02 00:22:42,RT @thronesfacts: Conleth Hill was not happy w...
3,2019-06-02 00:22:39,DAMN-sa Stark. #gameofthrones #icon https://t....
4,2019-06-02 00:22:33,RT @Steve_Games: #DeadwoodTheMovie\nSeries Fin...
5,2019-06-02 00:22:19,RT @Steve_Games: Xceptionals\n#SciFi #amwritin...
6,2019-06-02 00:22:14,RT @thronesfacts: Sophie Turner gives therapy ...
7,2019-06-02 00:22:08,#GameofThrones' George R.R. Martin is Building...
8,2019-06-02 00:21:54,RT @Steve_Games: #inadifferentlife The Wild Pr...
9,2019-06-02 00:21:35,RT @GameOfThrones: Queen in the North.\nTake a...


## Correct date to datetime type

In [5]:
twitterFeed["datetime"] = pd.to_datetime(twitterFeed["datetime"])
twitterFeed

,datetime,tweet
0,2019-06-02 00:22:56,RT @isyuniarto: Which House are you?\nGOTZILLA...
1,2019-06-02 00:22:54,My 🔥#GameofThrones #Finale take: it was fuckin...
2,2019-06-02 00:22:42,RT @thronesfacts: Conleth Hill was not happy w...
3,2019-06-02 00:22:39,DAMN-sa Stark. #gameofthrones #icon https://t....
4,2019-06-02 00:22:33,RT @Steve_Games: #DeadwoodTheMovie\nSeries Fin...
5,2019-06-02 00:22:19,RT @Steve_Games: Xceptionals\n#SciFi #amwritin...
6,2019-06-02 00:22:14,RT @thronesfacts: Sophie Turner gives therapy ...
7,2019-06-02 00:22:08,#GameofThrones' George R.R. Martin is Building...
8,2019-06-02 00:21:54,RT @Steve_Games: #inadifferentlife The Wild Pr...
9,2019-06-02 00:21:35,RT @GameOfThrones: Queen in the North.\nTake a...


In [7]:
twt = twitterFeed["tweet"].iloc[0]
twt

'RT @isyuniarto: Which House are you?\nGOTZILLA House Sigils: Game of Thrones X Godzilla X Javanese Wayang Style. Art by https://t.co/AaxDF9i…'

In [8]:
twitterFeed.dtypes

datetime    datetime64[ns]
tweet               object
dtype: object

## Getting the non-repeated tweets

In [10]:
uniqueTweets = twitterFeed["tweet"].unique()
uniqueTweets

array(['RT @isyuniarto: Which House are you?\nGOTZILLA House Sigils: Game of Thrones X Godzilla X Javanese Wayang Style. Art by https://t.co/AaxDF9i…',
       'My 🔥#GameofThrones #Finale take: it was fucking boring. 😴',
       'RT @thronesfacts: Conleth Hill was not happy when the script revealed Varys’ death #GameofThrones https://t.co/HnHIqLmiW3',
       ...,
       '@nypost If my entire career would forever be linked to a dopey soyboy character in a show that went off the rails as badly as #GameOfThrones , I would seek solace in a bottle as well. Or my saucy redheaded wife who was fortunate to get killed off in the series before it sucked.',
       '@ETCanada @GameOfThrones Don’t get it, how stressful can it be to utter three lines throughout an entire season? 😀 #GameOfThrones',
       "That little smile in between the kiss...don't tell me that these two never dated 😏 BE STILL MY KIMILIA HEART ♥ \n#GameofThrones\n#TheLastWatch https://t.co/DGBqqlWrMw"],
      dtype=object)

In [11]:
len(uniqueTweets)

15682

In [12]:
uniqueTweets[0]

'RT @isyuniarto: Which House are you?\nGOTZILLA House Sigils: Game of Thrones X Godzilla X Javanese Wayang Style. Art by https://t.co/AaxDF9i…'

## Deleting RTs

In [14]:
isRT = [txt[0:3]!="RT " for txt in uniqueTweets]
isRT

[False,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 False,
 Fals

In [15]:
uniqueNonRT = uniqueTweets[isRT]
uniqueNonRT

array(['My 🔥#GameofThrones #Finale take: it was fucking boring. 😴',
       'DAMN-sa Stark. #gameofthrones #icon https://t.co/oqphgJ5ETo',
       "#GameofThrones' George R.R. Martin is Building a Psychedelic Theme Park https://t.co/j1VtDfBE9j https://t.co/P6KdsCVG7g",
       ...,
       '@nypost If my entire career would forever be linked to a dopey soyboy character in a show that went off the rails as badly as #GameOfThrones , I would seek solace in a bottle as well. Or my saucy redheaded wife who was fortunate to get killed off in the series before it sucked.',
       '@ETCanada @GameOfThrones Don’t get it, how stressful can it be to utter three lines throughout an entire season? 😀 #GameOfThrones',
       "That little smile in between the kiss...don't tell me that these two never dated 😏 BE STILL MY KIMILIA HEART ♥ \n#GameofThrones\n#TheLastWatch https://t.co/DGBqqlWrMw"],
      dtype=object)

In [16]:
len(uniqueNonRT)

10408

## Get the noun-phrases

In [18]:
p.set_options()

AttributeError: module 'preprocessor' has no attribute 'set_options'